In [1]:
import QDMPy.io as Qio
import QDMPy.fit as Qfit
import QDMPy.plot as Qplot
import QDMPy.bfield as Qbfield

import matplotlib
import matplotlib.pyplot as plt
import warnings

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget
# %matplotlib inline

### Load raw data

In [3]:
options_dict = {
    # Zyla: C:\ExperimentalData\2021\2021-02\2021-02-12_D432_SiPV-20
    "base_dir": r"C:\ExperimentalData\2021\2021-02\2021-02-12_D432_SiPV-20",
    "filepath": "ODMR - Pulsed with IR_28",

    "additional_bins": 2,

    "system_name": "Zyla",

    "ROI": "Full",
    "ROI_start": [10, 10],
    "ROI_end": [200,200],

    "AOI_1_start": [60, 145],
    "AOI_1_end": [70, 155],
    "AOI_2_start": [20, 20],
    "AOI_2_end": [24, 24],
    "AOI_3_start": [30, 20],
    "AOI_3_end": [32, 24],
    "single_pixel_check": [90, 150],

    "normalisation": "div",

    "fit_backend": "scipyfit",
    "fit_backend_comparison": ["scipyfit"],
    "fit_pixels": True,
    "force_fit": False,

    "use_ROI_avg_fit_res_for_all_pixels": True,

    "scipyfit_sub_threads": 2,

    "fit_functions":  {"linear": 1, "lorentzian": 8},

    "pos_guess": [2714, 2768, 2811, 2860, 2915, 2958, 2993, 3034],
    "pos_range": 25,

    "amp_guess": -0.005,
    "amp_bounds": [-0.0300, -0.0003],

    "fwhm_guess": 9,
    "fwhm_bounds": [5, 20],

    "c_guess": 1,
    "c_bounds": [0.9995, 1.0005],

    "m_guess": 0,
    "m_bounds": [-1e-6, 1e-6],

    "gpufit_tolerance": 1e-12,
    "gpufit_max_iterations": 25,
    "gpufit_estimator_id": "LSE",

    "save_fig_type": "png" # or try pdf
}

In [4]:
options = Qio.load_options(options_dict=options_dict, check_for_prev_result=True)
Qplot.set_mpl_rcparams(options)
image, prelim_sweep_list = Qio.load_image_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = Qio.reshape_dataset(
        options, image, prelim_sweep_list
    )
ROI_PL_fig = Qplot.plot_ROI_PL_image(options, PL_image)

AOI_PL_fig = Qplot.plot_AOI_PL_images(options, PL_image_ROI)

AOI_spectra_fig = Qplot.plot_AOI_spectra(options, sig, ref, sweep_list)

Qio.save_PL_data(options, PL_image, PL_image_ROI)

C:\Widefield_processing_code\QDMPy_git\QDMPy\io\raw.py:960: UserWarning: AOI_1 Rectangle starts outside image (too large in y), setting to zero.
  warnings.warn(f"{name} Rectangle starts outside image (too large in y), setting to zero.")
C:\Widefield_processing_code\QDMPy_git\QDMPy\io\raw.py:970: UserWarning: AOI_1 Rectangle too big in y, cropping to image.
  warnings.warn(f"{name} Rectangle too big in y, cropping to image.")
C:\Widefield_processing_code\QDMPy_git\QDMPy\io\raw.py:316: UserWarning: Avoiding IndexError for single_pixel_check (setting pixel check to centre of image):
index 150 is out of bounds for axis 1 with size 128
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fit AOI, ROI averages, single pixel

In [5]:
fit_model = Qfit.define_fit_model(options)
backend_ROI_results_lst = Qfit.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)
ROI_fit_fig = Qplot.plot_ROI_avg_fits(options, backend_ROI_results_lst)

fit_result_collection_lst = Qfit.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, backend_ROI_results_lst)
AOI_fit_fig = Qplot.plot_AOI_spectra_fit(options, sig, ref, sweep_list, fit_result_collection_lst, fit_model)

C:\Widefield_processing_code\QDMPy_git\QDMPy\fit\_scipyfit.py:36: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fit pixels

In [6]:
wanted_roi_result = next(filter(lambda result : result.fit_backend == options["fit_backend"], backend_ROI_results_lst))
if options["found_prev_result"]:
    pixel_fit_params = Qio.load_prev_fit_results(options)
    warnings.warn("Plotting previous fit results.")
elif options["fit_pixels"]:
    pixel_fit_params = Qfit.fit_pixels(options, sig_norm, sweep_list, fit_model, wanted_roi_result)
else:
    pixel_fit_params = None # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

Qio.save_pixel_fit_results(options, pixel_fit_params)

<ipython-input-6-746ba6ac9154>:4: UserWarning: Plotting previous fit results.
  warnings.warn("Plotting previous fit results.")


### Plot (fit) parameters

In [7]:
c_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "c")
m_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "m")
pos_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "pos")
amp_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "amp")
fwhm_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")
residual_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "residual")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Initial guess and fit vs data (flattened)

In [8]:
flat_pos_fig = Qplot.plot_params_flattened(options, fit_model, pixel_fit_params, wanted_roi_result, "pos")
flat_resid_fig = Qplot.plot_params_flattened(options, fit_model, pixel_fit_params, wanted_roi_result, "residual")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Load reference experiment - note, you can fit in this same notebook!

In [9]:
ref_options_dir = r"C:\ExperimentalData\2021\2021-02\2021-02-12_D432_SiPV-20\ODMR - Pulsed without IR_36_processed_bin_8" # if you want to use a reference experiment
ref_fit_params = Qio.load_reference_experiment_fit_results(options, ref_options=None, ref_options_dir=ref_options_dir)

### Calculate B_nvs, plot sig - ref

In [10]:
sig_bnvs, sig_dshifts = Qbfield.get_bnvs_and_dshifts(pixel_fit_params)
ref_bnvs, ref_dshifts = Qbfield.get_bnvs_and_dshifts(ref_fit_params)

# TODO other bsubs e.g. subtract_blurred, subtract_outside_polygons etc. should be defined on any 2D array, do that later
bnvs = Qbfield.bnv_refsub(options, sig_bnvs, ref_bnvs)

Qio.save_bnvs_and_dshifts(options, "sig", sig_bnvs, sig_dshifts)
Qio.save_bnvs_and_dshifts(options, "sig_sub_ref", bnvs, None)

bnv_sig_plot = Qplot.plot_bnvs_and_dshifts(options, "sig", sig_bnvs, sig_dshifts)
bnv_ref_plot = Qplot.plot_bnvs_and_dshifts(options, "ref", ref_bnvs, ref_dshifts)
bnvs_plot = Qplot.plot_bnvs_and_dshifts(options, "sig_sub_ref", bnvs, [])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Save and view options dict

In [11]:
Qio.save_options(options)
# print(Qio.dict_to_json_str(options))